In [3]:
from keras.datasets import mnist
import numpy as np
np.random.seed(7)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [9]:
def get_data_simpleNN(X_train, y_train, X_test, y_test):
    print("Shape before:", X_train.shape)
    num_pixels = X_train.shape[1] * X_train.shape[2]
    # Flatten the 2D array of 28x28 into 784 vector. Total shape goes from 60000, 28, 28 to 60000, 784
    X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
    X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')
    print("Shape after:", X_train.shape)

    # Normalise the input values. Currently it goes from 0 to 255.
    X_train = X_train / 255
    X_test = X_test / 255
    
    y_train = np_utils.to_categorical(y_train) # So 5 becomes [0, 0, 0, 0, 0, 1, 0, 0, 0, 0] since here 10 labels
    y_test = np_utils.to_categorical(y_test)
    num_classes = y_test.shape[1]
    
    return X_train, y_train, X_test, y_test, num_classes


def get_data_CNN(X_train, y_train, X_test, y_test):
    # For CNN the dimension should be [width][height][channes] for each input where [channels] is the number of colour channels 
    # in image. Here we use grayscale so it has value 1. This is the default formal. See argument data_format in Conv2D in Keras.
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
    X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')
    
    X_train = X_train / 255
    X_test = X_test / 255
    
    y_train = np_utils.to_categorical(y_train) # So 5 becomes [0, 0, 0, 0, 0, 1, 0, 0, 0, 0] since here 10 labels
    y_test = np_utils.to_categorical(y_test)
    num_classes = y_test.shape[1]
    
    return X_train, y_train, X_test, y_test, num_classes

In [10]:
def baseline_model_NN(num_classes):
    model = Sequential()
    # We have one unit per input. Only one hidden layer here
    model.add(Dense(num_pixels, input_dim=num_pixels, activation='relu', kernel_initializer='normal'))
    # Output layer where we use one unit per class label. Use of softmax activation gives a probability distribution over outputs
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def baseline_model_CNN(num_classes):
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [11]:
X_train_CNN, y_train_CNN, X_test_CNN, y_test_CNN, num_classes = get_data_CNN(X_train, y_train, X_test, y_test)
model = baseline_model_CNN(num_classes)
model.fit(X_train_CNN, y_train_CNN, validation_data=(X_test_CNN, y_test_CNN), epochs=10, batch_size=200, verbose=2)
scores = model.evaluate(X_test_CNN, y_test_CNN, verbose=0)

print("Error:", 100-scores[1]*100)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
 - 10s - loss: 0.2440 - acc: 0.9311 - val_loss: 0.0739 - val_acc: 0.9775
Epoch 2/10
 - 5s - loss: 0.0705 - acc: 0.9789 - val_loss: 0.0525 - val_acc: 0.9822
Epoch 3/10
 - 5s - loss: 0.0513 - acc: 0.9848 - val_loss: 0.0400 - val_acc: 0.9873
Epoch 4/10
 - 5s - loss: 0.0384 - acc: 0.9887 - val_loss: 0.0393 - val_acc: 0.9868
Epoch 5/10
 - 5s - loss: 0.0316 - acc: 0.9906 - val_loss: 0.0330 - val_acc: 0.9886
Epoch 6/10
 - 5s - loss: 0.0281 - acc: 0.9914 - val_loss: 0.0352 - val_acc: 0.9871
Epoch 7/10
 - 5s - loss: 0.0216 - acc: 0.9934 - val_loss: 0.0359 - val_acc: 0.9887
Epoch 8/10
 - 5s - loss: 0.0187 - acc: 0.9937 - val_loss: 0.0313 - val_acc: 0.9902
Epoch 9/10
 - 5s - loss: 0.0155 - acc: 0.9951 - val_loss: 0.0330 - val_acc: 0.9896
Epoch 10/10
 - 5s - loss: 0.0134 - acc: 0.9957 - val_loss: 0.0340 - val_acc: 0.9893
Error: 1.07
